In [1]:
import sys 
sys.path.append('../')
import pandas as pd 
import numpy as np 
import talib
import sqlite3 as sql
import matplotlib.pyplot as plt 
from backtester import * 
from fxcmtoken import my_assets

In [2]:
for i, a in enumerate(my_assets):
    print(i, a)

0 EUR/USD
1 USD/JPY
2 GBP/USD
3 USOil
4 SPX500
5 VOLX
6 XAU/USD
7 Bund
8 BTC/USD


In [36]:
conn = sql.connect('../PriceData/PriceData_m5.db')
ticker = my_assets[8]
print(ticker)
data_sql = pd.read_sql(f"SELECT * FROM '{ticker}'", conn, parse_dates=['date'], index_col=['date'])
for col in ['open', 'close', 'high', 'low']:
    data_sql[col] = data_sql[['bid'+col, 'ask'+col]].mean(axis=1)
conn.close()
data_sql.tail()

BTC/USD


,bidopen,bidclose,bidhigh,bidlow,askopen,askclose,askhigh,asklow,tickqty,open,close,high,low
date,,,,,,,,,,,,,
2022-07-08 20:30:00,21909.0,21933.2,21963.0,21865.0,21949.0,21973.2,22003.0,21905.0,571,21929.0,21953.2,21983.0,21885.0
2022-07-08 20:35:00,21932.8,21839.1,21993.9,21825.6,21972.8,21879.1,22033.9,21865.6,777,21952.8,21859.1,22013.9,21845.6
2022-07-08 20:40:00,21839.5,21859.5,21877.0,21788.4,21879.5,21899.5,21917.0,21828.4,584,21859.5,21879.5,21897.0,21808.4
2022-07-08 20:45:00,21860.6,21827.1,21860.6,21791.2,21900.6,21867.1,21900.6,21831.2,539,21880.6,21847.1,21880.6,21811.2
2022-07-08 20:50:00,21825.3,21843.6,21844.8,21798.7,21865.3,21863.6,21884.8,21838.7,408,21845.3,21853.6,21864.8,21818.7


In [37]:
num_periods =  [i for i in np.arange(0.1, 1, 0.1)]

for n_period in num_periods:
    data = data_sql.copy(deep=True)
    print(ticker)
    print(f"Period: {n_period:.2f}")
    
    data['ht'] = talib.HT_TRENDLINE(data.close)
    data.dropna(axis=0, inplace=True)
    data['mama'], data['fama'] = talib.MAMA(data.ht, fastlimit=n_period, slowlimit=n_period/10)
    data.dropna(axis=0, inplace=True)
    data['signals'] = np.where(data.mama > data.fama, 1, -1) 
    backtest = IterativeBacktester(data=data,
                                   signals=data.signals,
                                   freq='m5')

    backtest.backtest(progress_bar=False)
    # backtest.results()
    print(f"{backtest.return_df.loc['TotalReturn', 'Portfolio']*100:.4f}%")
    
    print("Signals")
    print(backtest.signals.value_counts())

    print("Signals change")
    print(backtest.signals.diff(1).dropna().apply(np.abs).value_counts())
    
    print("-"*50)

BTC/USD
Period: 0.10
-29.3498%
Signals
-1    4980
 1    4925
Name: signals, dtype: int64
Signals change
0.0    9832
2.0      72
Name: signals, dtype: int64
--------------------------------------------------
BTC/USD
Period: 0.20
-16.3125%
Signals
-1    5212
 1    4693
Name: signals, dtype: int64
Signals change
0.0    9786
2.0     118
Name: signals, dtype: int64
--------------------------------------------------
BTC/USD
Period: 0.30
14.3593%
Signals
-1    5195
 1    4710
Name: signals, dtype: int64
Signals change
0.0    9744
2.0     160
Name: signals, dtype: int64
--------------------------------------------------
BTC/USD
Period: 0.40
13.6408%
Signals
-1    5194
 1    4711
Name: signals, dtype: int64
Signals change
0.0    9716
2.0     188
Name: signals, dtype: int64
--------------------------------------------------
BTC/USD
Period: 0.50
2.2731%
Signals
-1    5151
 1    4754
Name: signals, dtype: int64
Signals change
0.0    9686
2.0     218
Name: signals, dtype: int64
--------------------